In [7]:
import requests

url = 'https://bina.az/baki/alqi-satqi/menziller'
api_key = 'a418ec7a1656038d81e4409b0e575eff'  # Replace with your ScraperAPI key  
proxy_url = f'http://api.scraperapi.com?api_key={api_key}&url={url}'

response = requests.get(proxy_url)

print(response.status_code)
print(response.text[:500]) 

200
<!DOCTYPE html><html lang="az"><head><title>Mənzil almaq, Bakı - bina.az</title><meta charset="UTF-8" /><meta content="Azərbaycanda pulsuz daşınmaz əmlak elanları - bina.az. Mənzil almaq, Bakı kateqoriyasında bütün elanlar" name="description" /><link href="https://bina.azstatic.com/assets/favicons/favicon-192x192-b40ea6169e17d157d4e6943453ee0f32374348b53abc40010d2ff8c81a2263ec.png" rel="apple-touch-icon" sizes="192x192" /><link href="https://bina.azstatic.com/assets/favicons/favicon-192x192-b40e


In [74]:
scrape_listings(first_page_url)

Etracted 28 total listings from https://bina.az/baki/kiraye/menziller


True

In [9]:
from bs4 import BeautifulSoup
import requests


html_content=response.text
soup=BeautifulSoup(html_content,"html.parser")

href=soup.find_all('a' ,href=True)
href


first_page_url='https://bina.az/baki/alqi-satqi/menziller'
base_url='https://bina.az/baki/alqi-satqi/menziller?page='

current_page=2
max_page=200
linkdata=set()


def scrape_listings(url):
    
    api_key = 'a418ec7a1656038d81e4409b0e575eff'  # Replace with your ScraperAPI key
    proxy_url = f'http://api.scraperapi.com?api_key={api_key}&url={url}'

    response = requests.get(proxy_url)

# checking url status 
    if response.status_code != 200:
        print(f"Failed fetch url {url} ,stopping")
        return False


        
    
    soup=BeautifulSoup(response.text,"html.parser")

    href=soup.find_all('a' ,href=True)


    new_linkset=set()
    for link in href:
        linkset=link['href']
        if '/items/' in linkset and all(x not in linkset for x in ['=','all','new']):
            new_linkset.add('https://bina.az'+linkset)

    linkdata.update(new_linkset)
            
    print(f"Etracted {len(new_linkset)}  listings from {url} . total listenings {len(linkdata)}")
    return True

scrape_listings(first_page_url)

while current_page<=max_page:
    page_url=f"{base_url}{current_page}"
    success=scrape_listings(page_url)


    if not success:
        break

    current_page +=1


print(f"\n✅ Scraping finished! Total unique listings extracted: {len(linkdata)}")



     


Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller . total listenings 28
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=151 . total listenings 56
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=152 . total listenings 84
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=153 . total listenings 111
Etracted 27  listings from https://bina.az/baki/alqi-satqi/menziller?page=154 . total listenings 138
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=155 . total listenings 165
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=156 . total listenings 192
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=157 . total listenings 220
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=158 . total listenings 248
Etracted 28  listings from https://bina.az/baki/alqi-satqi/menziller?page=159 . total listenings 275
Et

In [29]:
seen=set()
dublicates=set()

for links in linkdata2:
    if links in seen:
        dublicates.add(links)
    else :
        seen.add(links)

if dublicates:
    print('Dublicate link found' ,dublicates)
else :
    print('No Dublicate')


No Dublicate


In [8]:
# create datafram from linkdata

In [23]:
import pandas as pd
import numpy as np
import re

Proporties_info=pd.DataFrame(columns=['Category','Area_m2','Floor','Building_Floor_count','Room_Count','Price','Currency','Location','Repair','Distirict'])
rows = []


for itm in linkdata:
    try: 

        api_key = 'a418ec7a1656038d81e4409b0e575eff'  # Replace with your ScraperAPI key
        proxy_url = f'http://api.scraperapi.com?api_key={api_key}&url={itm}'

        response = requests.get(proxy_url)

        if response.status_code != 200:
            print(f"Failed to fetch {itm}")

        soup = BeautifulSoup(response.text, "html.parser")

        
        kiraye=soup.find_all('div' , class_= 'product-properties__column')
        price=soup.find_all('div',class_='product-price')
        location=soup.find_all('ul' , class_='product-extras bz-d-flex bz-align-center bz-gap-15 bz-wrap-wrap')
        lct=soup.find_all('a' ,class_='open_map')
        ozellikler= re.sub('<[^>]+>', ' ', str(kiraye)).split('  ')  # neticeye bax orda '' isaresi niye silmek olmur
        loc=re.sub('<[^>]+>',' ',str(location)).split('  ') 
        lc=re.sub('<[^>]+>' , ' ', str(lct)).split('  ')
        price=re.sub('<[^>]+>', ' ' ,str(price)).split ('  ')


        if len(ozellikler)>14 and len(price)>3:
            kategoriya=ozellikler[2]
            sahe=ozellikler[8].replace('m2','')
            mertebe=ozellikler[5]
            otag_sayi=ozellikler[11]
            qiymet=price[1].replace(' ','')
            kira_novu=price[3].replace('/','')
            valyuta=price[2]
            erazi=str(loc[1:-1]).replace('m2','')
            temir=ozellikler[14]
            Rayon=lc[0].split(', ')
            District=Rayon[1].replace('r.','') if len(Rayon) > 1 else np.nan



            new_row = {
                'Category': kategoriya,
                'Area_m2': sahe,
                'Floor': mertebe.split('/')[0],
                'Building_Floor_count' : mertebe.split('/')[1],
                'Room_Count': otag_sayi,
                'Price': qiymet,
                'Currency': valyuta,
                'Location': erazi,
                'Repair': temir,
                'Distirict' : District
            } 


            rows.append(new_row)
            print(f"Extracted link count : {itm}")
            
            # Add the new row to the list

           



    except Exception as e: 
        print(e)

       
        
        

if rows:
    df = pd.concat([Proporties_info, pd.DataFrame(rows)], ignore_index=True)
    print(Proporties_Data)

           





Extracted link count : https://bina.az/items/5004576
Extracted link count : https://bina.az/items/4936652
Extracted link count : https://bina.az/items/5015922
Extracted link count : https://bina.az/items/4894081
Extracted link count : https://bina.az/items/4808605
Extracted link count : https://bina.az/items/5074158
Extracted link count : https://bina.az/items/5091922
Extracted link count : https://bina.az/items/4621986
Extracted link count : https://bina.az/items/5074794
Extracted link count : https://bina.az/items/4404201
Extracted link count : https://bina.az/items/5068771
Extracted link count : https://bina.az/items/4952660
Extracted link count : https://bina.az/items/5077973
Extracted link count : https://bina.az/items/5107895
Extracted link count : https://bina.az/items/5049499
Extracted link count : https://bina.az/items/5095426
Extracted link count : https://bina.az/items/5066913
Extracted link count : https://bina.az/items/5072364
Extracted link count : https://bina.az/items/5

#Explore dataset

In [31]:
print(f'Dataframe shape : {df.shape} ')

print('Null values count by columns')

print(df.isnull().sum())

print(' Datframe columns descriptions')

print(df.info())

In [ ]:

def avg_price (row):

    if row['Price']<=100000:
        return 100000
    elif row['Price'] >100000 and row['Price'] <=200000:
        return 200000
    elif row['Price']>200000 and row['Price']<=300000:
        return 300000
    elif row['Price']>300000 and row['Price']<=500000:
        return 500000
    elif row['Price']>500000 and row ['Price']<=700000:
        return 700000
    elif row['Price']>700000 and row ['Price']<=1000000:
        return 1000000
    else:
        return +1000000
    


In [ ]:
df['Avg_Price'] = df.apply(avg_price, axis=1)
df.head(10)

Distirubition of numeric columuns accros to the Price values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

column_list=['Category','Floor','Room_Count','Repair','Price']

def histogram_columns(data):

    filtered_columns=[col for col in column_list if col !='Price']

    for clmn in filtered_columns:
        plt.figure(figsize=(8,4))
        sns.kdeplot(data=data,x='Price' ,hue=clmn ,shade=True)
        
        plt.title(f'Distribution of {clmn} according to Price')
        plt.xlabel(clmn)
        plt.ylabel('Density')
        plt.show()
histogram_columns(df)

In [ ]:
#explore all columns by histogram
def histogram_of_columns(data):
    
    for clmn in data.columns:
        plt.figure(figsize=(8,4))
        sns.histplot(data[clmn] ,kde=False)
        plt.title(clmn)
        plt.xlabel(clmn)
        plt.ylabel('count')
        plt.show()
    

histogram_of_columns(df)

Cleaning miss values  and Modifying columuns

#Cleaning miss values and modifying columuns


In [ ]:
# remove m2 in Area_m2 column

df['Area_m2']=df['Area_m2'].str.replace(r'\D+' , '',regex=True).astype(int)


# Fill the null values in Distirict columns  with values which contains in Location 

def location_clmn(row):
    # Only process rows where Distirict is NaN
    if pd.fisna(row['Distirict']):
        # Split Location by comma (handle multiple parts)
        location_parts = row['Location'].split(',')
        
        # Iterate through each part of Location
        for loc in location_parts:
            if ' r.' in loc:  # Check if 'r' is in the location part
                # Return modified location (remove 'r' and strip spaces)
                cleaned = loc.replace(' r.', '').replace(' r.]', '').replace(']' , '').replace('[' , '' ).replace(' r', '').replace("'" , '').strip()

                

                return cleaned
        
        # If no part of Location contains 'r', return 'Unknown'
        return 'Unknown'
    
    # If Distirict is not NaN, return the original value
    return row['Distirict']

df['New District']=df.apply(location_clmn,axis=1)

In [ ]:
#Create top 5 District 
df.columns
df['New District']=df['New District'].str.replace(' ' , '').str.strip()
#df['New District'].unique()
top_5_distirict=df['New District'].value_counts().head(5).index.tolist()
df['District_5']=df['New District'].apply(lambda x : x if x in top_5_distirict else 'Other' )
df['District_5'].unique()
# Remove some useless columns
df.drop(columns=['Currency','Location','Distirict','New District'],inplace=True)

In [ ]:
print(df.head(10))
print(df.info())
print(df.isnull().sum())

In [ ]:
#Detect columns which are has outliers 

columun_list=['Area_m2','Floor','Building_Floor_count','Room_Count','Price']

for i in df_dummies[columun_list] :
    plt.figure(figsize=(10,6))
    sns.boxplot(x=df_dummies[i])
    plt.title(i)
    plt.show()

In [ ]:
# Detect Outliers on price columun
# Considering leaner regression model therefore would better  in order to  remove outliers in target 
# remove Target outliers (Price)

print(f'Before remove outliers : {df_dummies.shape}')
Q1=df_dummies['Price'].quantile(0.25)
Q3=df_dummies['Price'].quantile(0.75)
IQR=Q3-Q1
df_without_out=df_dummies[df_dummies['Price']<=(Q3+1.5*IQR)]
price_outliers

print(f'After remove outliers : {df_without_out.shape}')



One-Hot encoding

In [ ]:
#Transform some columuns as Dummy (One-hot encoding)
df_dummies=pd.get_dummies(df,columns=['Category','Repair','District_5'],drop_first=True,dtype='int')
df_dummies

Corrolation Test

In [ ]:
# Corralation test

correlation_matrix=df_dummies.corr()


plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix , annot=True , cmap='coolwarm' )
plt.title('Correlation Matrix')
plt.show()



In [ ]:
# cheking regression between target and independent columns 

cols=[ 'Area_m2','Floor' ,'Building_Floor_count' ,'Room_Count']
plt.figure(figsize=(20,30))



for i in range(len(cols)):
    plt.subplot(5, 3, i + 1)
    plt.title(cols[i] + ' - Price')
    
    
    sns.regplot(x=df_dummies[cols[i]], y=df_dummies.Price)

plt.tight_layout()
plt.show()

Split Train-test

In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop(columns=['Price','Avg_Price'])
y=df['Price']

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=42)



print(f'X_train :{X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train :{y_train.shape}')
print(f'y_test :{y_test.shape}')

In [ ]:
# Demonstrate 3 type of model in order to decide best one 

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define the models in a dictionary
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(random_state=42)
}

# Loop through each model and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"🔹 {name}")
    print(f"   ✅ MSE: {mse:.2f}")
    print(f"   ✅ R² Score: {r2:.4f}")
    print("-" * 40)